In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices())
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import numpy as np
import pandas as pd

import keras as k

In [ ]:
train_set = pd.read_csv('C:/Users/thanawin/Desktop/Deep/pixel_trainset2.csv')
val_set = pd.read_csv('C:/Users/thanawin/Desktop/Deep/pixel_trainset2.csv')
#test_set = pd.read_csv('C:/Users/ROG/Downloads/pixel_testset2.csv')

In [ ]:
y_train=train_set['label']

In [ ]:
X_train = np.array(list(map(str.split, train_set.pixel)), np.float32)

X_val = np.array(list(map(str.split, val_set.pixel)), np.float32)
#X_test = np.array(list(map(str.split, test_set.pixel)), np.float32) 
Y_val=val_set['label']
#_test=test_set['label']

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1) 
X_val = X_val.reshape(X_val.shape[0], 128, 128, 1)
#X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
Y_val.shape

In [ ]:
y_train = to_categorical(y_train, 7)
Y_val = to_categorical(Y_val, 7)

In [ ]:
#CNN
cnn4 = Sequential() #Sequential model allows to build a model layer by layer

#--------------------------------Input layer------------------------------------------# 
#COnvolution layer parametrs:
#No of output filters: 32
#kernel_size: specifying the height and width of the 2D convolution window : (3,3)
#activation function : relu
#input_shape : shape of image matrix : 48x48x1 : (Gray Scale Image)
cnn4.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(128,128,1))) 
#Batch Normalization : It is used to normalize the output of the previous layers. The activations scale the input layer in normalization.
#Using batch normalization learning becomes efficient also it can be used as regularization to avoid overfitting of the model.  
cnn4.add(BatchNormalization())

#--------------------------------Second layer------------------------------------------# 
cnn4.add(Conv2D(8, kernel_size=(3, 3), activation='relu')) 
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2))) #Down-sampling opertaion to reduce dimensionality of feature map
cnn4.add(Dropout(0.25)) #regularization technique to reduce overfitting: dropouts randomly switches off some neurons in the network which forces the data to find new paths

#--------------------------------Third layer------------------------------------------# 
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

#--------------------------------Fourth layer------------------------------------------# 
cnn4.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

#--------------------------------five layer------------------------------------------# 
cnn4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

#--------------------------------six layer------------------------------------------# 
cnn4.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Flatten()) #converts 3d vector to 1 d vector : Flatten serves as a connection between the convolution and dense layers.

#--------------------------------Output layers------------------------------------------# 
#‘Dense’ is the layer type we will use in for our output layer. 
cnn4.add(Dense(256, activation='relu')) #Dense layer predict labels
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(256, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(7, activation='softmax'))        #FINAL PREDICTION
#Softmax makes the output sum up to 1 so the output can be interpreted as probabilities.

In [ ]:
#Parameters:
#1) The optimizer controls the learning rate. Optmizerused : 'adam'. The adam optimizer adjusts the learning rate throughout training.
#2) Loss function used: ‘categorical_crossentropy". A lower score indicates that the model is performing better.
#3) To make things even easier to interpret, ‘accuracy’ metric is used to see the accuracy score on the validation set when we train the model.
cnn4.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
#The batch size defines the number of samples that will be propagated through the network.(at a time)
cnn4.fit(X_train,
         y_train,
         batch_size=32,
         epochs=30,
         verbose=1,
         validation_data=(X_val, Y_val)
         )

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,6))
plt.plot(cnn4.history.history['accuracy'], label='Train accuracy')
plt.plot(cnn4.history.history['val_accuracy'], label='Validation accuracy')
plt.ylabel('Value')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
cnn4.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(cnn4, to_file='model.png')

In [ ]:
pip install pydot